<h1>Redes RBF - Exercicio</h1>

In [5]:
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np
import torch


from sklearn.datasets import load_boston
boston = load_boston()
values_X = pd.DataFrame(load_boston().data, columns = boston.feature_names).values # importando as 13 variáveis
values_Y = pd.DataFrame(boston.target).values #importando valores medios das casas ocupadas

#Calculando a porcentagem do dataset que virá ser utilizado
aux = values_X[:,1].shape
N_total = aux[0]
N = int(.8*N_total)

#Pegando N linhas e atriuindo a uma varivel para treinamento
x = values_X[:N,:]
y_train = torch.from_numpy(values_Y[:N,:])
bias = np.ones(y_train.shape)
x_train= torch.from_numpy(np.concatenate((bias,x), axis=1))

#Pegando o linhas a partir de N para variaveis de test
test_y = torch.from_numpy(values_Y[N:,:])
test_y.shape
biasTest = np.ones(test_y.shape)
values_testX = values_X[N:,:]
test_X = torch.from_numpy(np.concatenate((biasTest,values_testX), axis=1))

#Treinamento da regressão
theta = torch.inverse(x_train.T @ x_train) @ x_train.T @ y_train
test = test_X @ theta #Teste
error = torch.mean((test_y - test )**2)#Calculo do erro quadratico medio

print(error)

tensor(32.7999, dtype=torch.float64)


<h4>Para implementacao do RBF primeiro escolhemos aleatoriamente os Centroides </h4> 

$$ X = \boxed{x^T_1\\x^T_2\\x^T_3\\\vdots\\x^T_n} \Longrightarrow C = \boxed{x^T_1\\x^T_4\\\vdots\\x^T_m} \Longrightarrow C = \boxed{c^T_1\\c^T_4\\\vdots\\c^T_m}_m\\Levando\ em\ conta\ que\ m<n$$

<h4>Formando minha matriz Z</h4>

$$Z = \begin{bmatrix}
||\vec x_1-\vec c_1||^2 && ||\vec x_1-\vec c_2||^2  && \cdots && ||\vec x_1-\vec c_m||^2\\
||\vec x_2-\vec c_1||^2  && ||\vec x_2-\vec c_2||^2  && \cdots && ||\vec x_2-\vec c_m||^2\\ 
\vdots && \vdots && \cdots && \vdots \\
||\vec x_n-\vec c_1||^2 && ||\vec x_n-\vec c_2||^2  && \cdots && ||\vec x_n-\vec c_m||^2
\end{bmatrix}_{NxM}$$

 $$\tilde{Z} \leftarrow exp 	\left \{ -\gamma Z\right \} $$

# <h4>Aplicando modelo linear<h4>

$$\hat{\vec \theta_{LS}} = (\tilde{Z}^T\tilde{Z})^{-1}\tilde{Z}\vec y $$

In [90]:
from numpy import random
numero_centroides = random.randint(N ,size=int(.2*N))
C = x[numero_centroides]

gamma = 1e-3
Z = torch.cdist(torch.from_numpy(x),torch.from_numpy(C))
Z = Z**2
kernel = torch.exp(-gamma*Z)
kernel.shape
kernel
boolBias = False


torch.Size([404, 80])


In [93]:
aux = kernel[:,1].shape
N_linhas_Bias = aux[0]

if(boolBias==False):
    Bias_RBF = torch.ones(N_linhas_Bias).reshape(-1,1)
    kernel = torch.cat((Bias_RBF,kernel),1)
    boolBias=True
    
theta_LS = torch.inverse(kernel.T @ kernel) @ kernel.T @ y_train
theta_LS.shape

torch.Size([81, 1])

In [92]:
aux = values_testX[:,1].shape
N_linhas_bias_teste = aux[0]
centroides_teste = random.randint(N_linhas_bias_teste ,size=int(.2*N_linhas_bias_teste))
centroides = values_testX[centroides_teste]
Z_teste = torch.cdist(torch.from_numpy(values_testX),torch.from_numpy(centroides))
Z_teste = Z_teste**2 
dataTest = torch.exp(-gamma*Z_teste)
dataTest.shape
#teste = datdaTest @ theta_LS

torch.Size([102, 20])